In [142]:
import numpy as np
import pandas as pd

# 1. Tout d'abord, une analyse de Fight Club

## 1.1 Lire les données

In [143]:
df = pd.read_csv("./data/fightclub_critiques.csv")
df.head() # Visualiser les données

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721


In [144]:
df.shape # 1000 commentaires observés

(1000, 11)

In [145]:
df['review_content'].head() # caractéristique cible

0    Fight Club sort à la fin 1999 sur les écrans. ...
1    Tout a été déjà dit sur le film de David Finch...
2    <p>Qu'une oeuvre aussi folle, aussi inconforta...
3    <p>Beaucoup pensent que ce film n'est qu'un él...
4    <p>Objet d’un véritable culte, Fight Club a ma...
Name: review_content, dtype: object

J'ai remarqué que les commentaires sont principalement en français et contiennent des caractères spéciaux, ce qui nécessite un prétraitement des commentaires.

## 1.2 Data Cleaning

In [146]:
import re
from bs4 import BeautifulSoup
from html import unescape


# Supprimer HTML
def remove_html(data):
    html_tag = re.compile(r'<.*?>')
    data = html_tag.sub('', data)
    return data

# Supprimer les URL (liens commençant par http://, https:// ou www.)
def remove_url(data):
    url_pattern = re.compile(r'(https?://\S+|www\.\S+)', re.IGNORECASE)
    data = url_pattern.sub('', data)
    return data

# Supprimer email
def remove_email(data):
    mail_pattern = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
    data = mail_pattern.sub('', data)
    return data

# Supprimer caractères spéciaux
def remove_special_char(data):
    punct_pattern = re.compile(r"[^a-zàâäçéèêëîïôöùûüÿñœæ'’\- ]+", re.IGNORECASE)
    data = punct_pattern.sub(' ', data)
    return data

# Fusionner les espaces redondants
def clean_spaces(data):
    return re.sub(r'\s+', ' ', data).strip()

# appliquer cleaning
df['review_content'] = df['review_content'].astype(str)             
df['review_content'] = df['review_content'].apply(remove_html)      
df['review_content'] = df['review_content'].apply(remove_url)       
df['review_content'] = df['review_content'].apply(remove_email)     
df['review_content'] = df['review_content'].apply(remove_special_char)  
df['review_content'] = df['review_content'].apply(clean_spaces)     
df['review_content'] = df['review_content'].str.lower()

print("Échantillons nettoyés：", df['review_content'])


Échantillons nettoyés： 0      fight club sort à la fin sur les écrans le fil...
1      tout a été déjà dit sur le film de david finch...
2      qu'une oeuvre aussi folle aussi inconfortable ...
3      beaucoup pensent que ce film n'est qu'un éloge...
4      objet d’un véritable culte fight club a marqué...
                             ...                        
995    bonjour regardez ce film car oui excellent mei...
996    ce film est très surcoté il as tout pour etre ...
997                                               banger
998                                                     
999    nous ne sommes que des produits de consommatio...
Name: review_content, Length: 1000, dtype: object


## 1.3 Vectorisation

Utiliser le TF-IDF modèle le plus traditionnel 

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
french_stop = stopwords.words('french')

vectorizer = TfidfVectorizer(
    stop_words=french_stop,  
    ngram_range=(1,2),        
    min_df=2,
    max_df=0.9
)

X = vectorizer.fit_transform(df['review_content'])

print("TF-IDF shape:", X.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zhous\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TF-IDF shape: (1000, 14534)


## 1.4 Calculer la similarité cosinus

En calculant la similarité cosinus, on peut identifier et recommander plusieurs autres commentaires présentant la plus grande similarité.

In [159]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

################ Utiliser la génération assistée par l'IA ########################

def topk_similar_by_index(X, texts: pd.Series, idx: int, k: int = 5):
    """Fournir les k commentaires les plus similaires au k-ième commentaire (à l'exclusion du commentaire lui-même)."""
    v = X[idx]
    sims = cosine_similarity(v, X).ravel()
    sims[idx] = -1.0 
    top = sims.argsort()[::-1][:k]
    return [(int(i), float(sims[i]), texts.iloc[i]) for i in top]

def basic_clean(text: str) -> str:
    if text is None:
        text = ""
    text = str(text)
    text = remove_html(text)
    text = remove_url(text)
    text = remove_email(text)
    text = remove_special_char(text)
    return text


def topk_similar_by_query(vec: TfidfVectorizer, X, texts: pd.Series, query: str, k: int = 5):
    """Renvoie les k commentaires les plus similaires pour tout texte de requête donné."""
    q_clean = basic_clean(query)
    q_vec = vec.transform([q_clean])
    sims = cosine_similarity(q_vec, X).ravel()
    top = sims.argsort()[::-1][:k]
    return [(int(i), float(sims[i]), texts.iloc[i]) for i in top]

# similarité basée sur un indice
idx=90  # Index dans le film (de 0 à 999)
print(f"\nCommentaire de référence n°{idx} :")
print(df['review_content'].iloc[idx][:100] + "...")
print("\nCritiques les plus similaires :")
for i, s, t in topk_similar_by_index(X, df['review_content'], idx, k=3):
    print(f"[{i}] sim={s:.3f} | {t}")

#similarité basée sur le texte de la requête
query = "incroyable acteur et musique touchante"
print(f"\nDemande de renseignements：{query}")
for i, s, t in topk_similar_by_query(vectorizer, X, df['review_content'], query=query, k=3):
    print(f"[{i}] sim={s:.3f} | {t}")
    
################ Utiliser la génération assistée par l'IA ########################


Commentaire de référence n°90 :
je n'ai pas vu ce film avant un long moment et l'on me faisait savoir que c'était une hérésie après ...

Critiques les plus similaires :
[858] sim=0.194 | je n'avais pas vu ce film avant la chute de fin originale
[989] sim=0.176 | j’ai été agréablement surprise par la fin d’où ma note cependant j’ai trouvé que c’était quand même long par moment
[906] sim=0.145 | vraiment ce film est ouf je ne m attendais pas à ça du tout et encore moins quand j ai commencé à le regarder surtout que je n avais aucune idée du plot je l ai lancé à l aveugle le début n est pas vraiment le style de film que j apprécie j ai eu un peu de mal car c est très spécial l atmosphère de ce film est vraiment particulière si ce n était pas pour le challenge je ne l aurais peut-être pas regardé en entier mais la fin a complètement changé la donne elle est très surprenante et incroyable mais j aurais pu me douter de cette révélation car il y a quelques indices et surtout je suis un adept

# 2. Analyse combinée Fight Club et Interstellar

Traiter simultanément les critiques de deux films, puis recommander des critiques similaires du même film, selon les besoins.

## 2.1 Lire les données interstellar

In [149]:
# lire les données interstellar
df_2 = pd.read_csv("./data/interstellar_critique.csv")
df_2.head() # Visualiser les données

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"<p>Chers lecteurs, chères lectrices,</p>\n\n<p...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,<p>Malgré ce que j'entends dire ou lis sur le ...,Tous les chemins mènent à l'Homme,347,blig,133793
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"<p>Un grand film, pour moi, c'est un film qui ...",Rage against the dying of the light.,453,Samu-L,41951
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,<p>Tout en n'étant absolument pas un admirateu...,Inspacetion,273,Nushku,24046


## 2.2 Marquer et fusionner deux films

In [150]:
df['movie'] = 'fightclub'
df_2['movie'] = 'interstellar'

In [151]:
# Fusionner les données
df_all = pd.concat([df, df_2], ignore_index=True)
print("Taille totale de l'échantillon après fusion：", len(df_all))

Taille totale de l'échantillon après fusion： 2000


## 2.3 Data cleaning

In [152]:
# Data cleaning utilisant les fonctions écrite dans la première partie
df_all['review_content'] = df_all['review_content'].astype(str)
df_all['review_content'] = df_all['review_content'].apply(remove_html)
df_all['review_content'] = df_all['review_content'].apply(remove_url)
df_all['review_content'] = df_all['review_content'].apply(remove_email)
df_all['review_content'] = df_all['review_content'].apply(remove_special_char)
df_all['review_content'] = df_all['review_content'].str.lower()
df_all['review_content']

0       fight club sort à la fin sur les écrans le fil...
1       tout a été déjà dit sur le film de david finch...
2       qu'une oeuvre aussi folle aussi inconfortable ...
3       beaucoup pensent que ce film n'est qu'un éloge...
4       objet d’un véritable culte fight club a marqué...
                              ...                        
1995    courez le voir    c'est pour moi le     l'odys...
1996    des effets spéciaux magnifiques  des paysages ...
1997    quel est le rôle de la science fiction   quell...
1998    christopher nolan nous livre encore une pépite...
1999    l'espace temps  la gravité  les trous noirs  l...
Name: review_content, Length: 2000, dtype: object

## 2.4 Vectorisation

In [153]:
# Vectorisation utilisant TDIDF
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
french_stop = stopwords.words('french')

vectorizer_all = TfidfVectorizer(
    stop_words=french_stop,  
    ngram_range=(1,2),        
    min_df=2,
    max_df=0.9
)

X_all = vectorizer_all.fit_transform(df_all['review_content'])

print("TF-IDF shape:", X_all.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zhous\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TF-IDF shape: (2000, 49121)


## 2.5 Calculer la similarité cosinus

In [160]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

################ Utiliser la génération assistée par l'IA ########################

# Sélectionner uniquement les critiques d’un film donné 
def subset_by_movie(df_all, X_all, movie_name):
    # Création d’un masque booléen pour isoler les lignes correspondant au film choisi
    mask = df_all['movie'].eq(movie_name).to_numpy()
    # Sous-ensemble de la matrice TF-IDF et des textes
    X_sub = X_all[mask]
    texts_sub = df_all.loc[mask, 'review_content'].reset_index(drop=True)
    return X_sub, texts_sub


# Trouver les critiques les plus similaires à partir d’un index dans un film
def topk_similar_by_index_subset(X_sub, texts_sub, idx, k=5):
    if not (0 <= idx < X_sub.shape[0]):
        raise IndexError(f"Index {idx} hors limites (0 à {X_sub.shape[0]-1}).")
    sims = cosine_similarity(X_sub[idx], X_sub).ravel()
    sims[idx] = -1.0
    top = sims.argsort()[::-1][:k]
    return [(int(i), float(sims[i]), texts_sub.iloc[i]) for i in top]


# Trouver les critiques les plus similaires à partir d’une requête texte
def topk_similar_by_query_subset(vec_all, df_all, X_all, movie_name, query, k=5):
    # Recherche uniquement dans le film spécifié, tout en utilisant le vocabulaire global (même espace TF-IDF)
    X_sub, texts_sub = subset_by_movie(df_all, X_all, movie_name)
    q = basic_clean(query)             
    q_vec = vec_all.transform([q])    
    sims = cosine_similarity(q_vec, X_sub).ravel()
    top = sims.argsort()[::-1][:k]
    return [(int(i), float(sims[i]), texts_sub.iloc[i]) for i in top]

# Choisir le film cible 
target = 'interstellar'   # modifier la valeur : 'fightclub' ou 'interstellar'

# trouver les critiques similaires à une critique donnée
X_sub, texts_sub = subset_by_movie(df_all, X_all, target)
idx_in_movie = 10  # Index dans le film (de 0 à 999)
print(f"\n[{target}] Critique n°{idx_in_movie} (texte de référence) :")
print(texts_sub.iloc[idx_in_movie][:100] + "...")
print("\nCritiques les plus similaires :")
for i, s, t in topk_similar_by_index_subset(X_sub, texts_sub, idx_in_movie, k=3):
    print(f"[{i}] sim={s:.3f} | {t}")

# trouver les critiques similaires à une requête libre
query = "incroyable musique et acteurs"
print(f"\n[{target}]Requête : {query}")
for i, s, t in topk_similar_by_query_subset(vectorizer_all, df_all, X_all, target, query, k=3):
    print(f"[{i}] sim={s:.3f} | {t}")
    
################ Utiliser la génération assistée par l'IA ########################



[interstellar] Critique n°10 (texte de référence) :
il y a vraiment de très bonnes choses dans cet interstellar  une brochettes d'acteurs impliqués dans...

Critiques les plus similaires :
[47] sim=0.114 | c'est presque l'épreuve incontournable de cette fin d'année   écrire quelque chose sur interstellar  a la lecture de nombreux articles  la moindre des choses que je puisse dire  c'est qu'il y a matière à faire couler l'encre à propos du film de christopher nolan  n'entretenons pas le suspens davantage   interstellar m'a relativement laissé sur le carreau   un produit intéressant mais chimérique de tout ce qui peut m'irriter dans le cinéma de nolan  et qui pourtant disposait de bien des bases pour être plutôt réussi  une fois n'est pas coutume  les lignes de ce papier contiendront de nombreux spoilers et sont donc  de préférence  réservées à ceux qui ont déjà vu ledit film  au passage  ce qui va suivre n'est pas tant une critique construite  mais plutôt un faisceau de pensées concern